In [1]:
import scipy.io
import numpy as np
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, initializers

In [2]:
#Cite this https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html

matlab_weights = scipy.io.loadmat('vgg_face.mat')
face_model = scipy.io.loadmat('face_model.mat')

In [3]:
net = matlab_weights['net']

In [4]:
normalization = net['normalization']
vgg_layers = net['layers']
classes = net['classes']

In [5]:
imageSize = normalization[0][0][0]['imageSize'][0][0]
border = normalization[0][0][0]['border'][0][0]
interpolation = normalization[0][0][0]['interpolation'][0][0]
averageImage = normalization[0][0][0]['averageImage'][0]
keepAspect = normalization[0][0][0]['keepAspect'][0][0][0]
imageSize, border, interpolation, averageImage, keepAspect

(array([224, 224,   3], dtype=uint8),
 array([32, 32], dtype=uint8),
 'bicubic',
 array([[129.18628],
        [104.76238],
        [ 93.59396]], dtype=float32),
 1)

In [6]:
names = classes[0][0][0]['name'][0][0]
descriptions = np.array([d[0] for d in classes[0][0][0]['description'][0].flatten()])
names, descriptions

(array([array(['n00000001'], dtype='<U9'),
        array(['n00000002'], dtype='<U9'),
        array(['n00000003'], dtype='<U9'), ...,
        array(['n00002620'], dtype='<U9'),
        array(['n00002621'], dtype='<U9'),
        array(['n00002622'], dtype='<U9')], dtype=object),
 array(['A.J._Buckley', 'A.R._Rahman', 'Aamir_Khan', ..., 'Zsa_Zsa_Gabor',
        'Zulay_Henao', 'Zuleyka_Silver'], dtype='<U33'))

In [7]:
model = models.Sequential()
for i, layer in enumerate(vgg_layers[0][0][0]):
    layer_type = layer['type'][0][0][0]
    layer_name = layer['name'][0][0][0]
    
    if layer_type == "conv":
        stride = layer['stride'][0][0][0][0]
        pad = layer['stride'][0][0][0][0]
        # learningRate = 1
        # weightDecay = 1
        #momentum = only for training
        
        #Wx + b
        W_matrix = initializers.Constant(layer['weights'][0][0][0][0])
        b = initializers.Constant(layer['weights'][0][0][0][1])
        shape = layer['weights'][0][0][0][0].shape
        
        filter_size = shape[0:2]
        num_filt = shape[3]
        if "fc" in layer_name:
            padding = "valid"
        else:
            padding = "same"
        
        if i == 0:
            model.add(layers.Conv2D(filters = num_filt, 
                                kernel_size = filter_size, 
                                input_shape = imageSize,
                                strides = (stride, stride),
                                padding = padding,
                                activation = "relu",
                                kernel_initializer = W_matrix,
                                bias_initializer = b))

        else:
            model.add(layers.Conv2D(filters = num_filt, 
                                kernel_size = filter_size,
                                strides = (stride, stride),
                                padding = padding,
                                activation = "relu",
                                kernel_initializer = W_matrix,
                                bias_initializer = b))
    elif layer_type == "pool":
        model.add(layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='same'))
    elif layer_type == "dropout":
        model.add(layers.Dropout(0.5))
    elif layer_type == "softmax":
        model.add(layers.Flatten())
        model.add(layers.Softmax())

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.build()

0
2
5
7
10
12
14
17
19
21
24
26
28
31
34
37


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [25]:
from PIL import Image
from PIL import ImageOps
I = Image.open("./ak.jpg")
#border = (250, 0, 250, 100) # left, up, right, bottom
border = (0, 0, 0, 122) #Aamir khan
I = ImageOps.crop(I, border)
I = I.resize((224, 224))
I.save('new.jpg')
I = np.array(I).astype(np.float)
I -= averageImage[0]
img = np.array([I])
print(img.shape)
res = model.predict(img)[0] * 100

(1, 224, 224, 3)


In [26]:
#import pandas as pd
#evaluation = pd.DataFrame(np.transpose([descriptions, res[0]]))
#evaluation
ranking = res.argsort()[-5:][::-1]
for rank in ranking:
    print(descriptions[rank], res[rank])

Aamir_Khan 69.78366
Adam_Driver 5.563041
D.B._Woodside 1.3797952
Manish_Dayal 1.3675587
John_Abraham 0.7590557
